<a href="https://colab.research.google.com/github/Erifai/Bi_Project/blob/main/Projet_BI_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Préparation de l'environnement :**

In [ ]:
#!pip install -U spacy 
#!pip install spacy-transformers
#!pip install spacy_entity_linker
#!python3 -m spacy_entity_linker "download_knowledge_base"
#!python -m spacy download en_core_web_sm
#!pip install -U sentence-transformers
#!sudo apt-get install metis
#!pip install Cluster_Ensembles

#à executer une seule fois et de redémarrer le runtime avec GPU 

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from spacy.lang.en.stop_words import STOP_WORDS
import string
import re
import operator
from functools import reduce
import nltk
#nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer


# **Chargement et nettoyage de données :**

In [64]:
#df = pd.read_fwf('/content/DBLP_Subset.txt')
#df=pd.read_table('/content/DBLP_Subset.txt',header=None)
#df = pd.read_csv("/content/DBLP_Subset.txt",sep="\n")
reading_file = open("/content/DBLP_Subset.txt", "r")

new_file_content = ""
count = 0
count_auth = 0
for line in reading_file:

  stripped_line = line.strip()

  new_line = stripped_line.replace(';',',')
  if "#t" in new_line :
    count_auth = 0
  if "#!" in new_line :
    count=0
  if "#@" in new_line :
    count_auth = 1
  new_line = new_line.replace('#c',';').replace('#*','\n;').replace('#c',';').replace('#t',';').replace('#index',';').replace('#!',';').replace('#@',';')
  if "#%" in new_line and count==0 :
    #print(new_line)
    new_line = new_line.replace('#%',';')
    #print(new_line)

    count =1
  if count_auth == 1 :
    new_line = new_line + " , "   

  new_file_content += new_line


reading_file.close()
df = pd.DataFrame([x.split(';') for x in new_file_content.split('\n')])
# .replace('#c',';').replace('#*',';').replace('#c%',';').replace('#t',';').replace('#index',';').replace('#!',';')
#   df = pd.DataFrame([x.split(';') for x in lines.split('\n')])

In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37964 entries, 0 to 37963
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       37964 non-null  object
 1   1       37963 non-null  object
 2   2       37963 non-null  object
 3   3       37963 non-null  object
 4   4       37963 non-null  object
 5   5       37963 non-null  object
 6   6       20875 non-null  object
 7   7       17852 non-null  object
dtypes: object(8)
memory usage: 2.3+ MB


In [66]:
df.rename(columns={0:"supprimer",1:"Titre",2:"Auteurs",3:"Année",4:"Revue",5:"IndexId",6:"IdRefrences",7:"Abstract"},inplace= True)
df.drop(columns=['supprimer'],inplace= True)
df.dropna(inplace= True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17852 entries, 1 to 37963
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Titre        17852 non-null  object
 1   Auteurs      17852 non-null  object
 2   Année        17852 non-null  object
 3   Revue        17852 non-null  object
 4   IndexId      17852 non-null  object
 5   IdRefrences  17852 non-null  object
 6   Abstract     17852 non-null  object
dtypes: object(7)
memory usage: 1.1+ MB


In [67]:
df.head()

,Titre,Auteurs,Année,Revue,IndexId,IdRefrences,Abstract
1,Improved Channel Routing by Via Minimization a...,"Chung-Kuan Cheng , David N. Deutsch ,",1988,DAC,131751,133716#%133521#%134343,Channel routing area improvement by means of v...
2,A fast simultaneous input vector generation an...,"Lei Cheng , Liang Deng , Deming Chen , Martin ...",2006,DAC,131752,132550#%530568#%436486#%134259#%283007#%134422...,Input vector control (IVC) technique is based ...
3,On the Over-Specification Problem in Sequentia...,"Kwang-Ting Cheng , Hi-Keung Tony Ma ,",1992,DAC,131756,455537#%1078626#%131745,The authors show that some ATPG (automatic tes...
4,Device and architecture co-optimization for FP...,"Lerong Cheng , Phoebe Wong , Fei Li , Yan Lin ...",2005,DAC,131759,214244#%215701#%214503#%282575#%214411#%214505...,Device optimization considering supply voltage...
5,Differential Fault Simulation - a Fast Method ...,"Wu-Tung Cheng , Meng-Lin Yu ,",1989,DAC,131760,131744#%806030,A newfast fault simulator called differential ...


In [ ]:
#df.to_csv("df.csv", sep=';', encoding='utf-8')

In [ ]:
#df = pd.read_csv("/content/df.csv", sep = ";",header=None, names=["Titre","Auteurs","Année","Publication","IndexId","IdRefrences","Abstract"])

In [ ]:
#df.info()

In [68]:
li = df['Revue'].value_counts()


In [69]:
list_revue = []
for i in range(0,len(li)-1):
  if(li[i] >= 50):
    list_revue.append(li.index[i])
  else:
    break


In [70]:
df_revue = pd.DataFrame(list_revue)
dff= pd.merge(df,df_revue,how='inner',left_on=['Revue'],right_on=[0])
dff.drop(columns=[0],inplace=True)
df = dff
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17429 entries, 0 to 17428
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Titre        17429 non-null  object
 1   Auteurs      17429 non-null  object
 2   Année        17429 non-null  object
 3   Revue        17429 non-null  object
 4   IndexId      17429 non-null  object
 5   IdRefrences  17429 non-null  object
 6   Abstract     17429 non-null  object
dtypes: object(7)
memory usage: 1.1+ MB


In [71]:
df['Revue'].value_counts()

DAC                                  2010
Electr. Notes Theor. Comput. Sci.    1891
IJCAI                                1891
ICCAD                                1666
SIGMOD Conference                    1529
GECCO                                1472
Discrete Applied Mathematics         1328
Journal of Systems and Software       899
ICIP                                  812
Des. Codes Cryptography               648
ICC                                   587
SIGMOD Record                         510
SMC                                   325
GECCO (Companion)                     269
Distributed Computing                 214
DATA BASE                             204
Data Min. Knowl. Discov.              175
ICCSA (1)                             152
ICCSA (2)                             145
ICCSA (3)                             144
IJCAI (1)                              82
ICCV                                   76
STOC                                   72
ICCBR                             

In [74]:
def downsample(df:pd.DataFrame, label_col_name:str) -> pd.DataFrame:
    # find the number of observations in the smallest group
    nmin = df[label_col_name].value_counts().max()
    return (df
            # split the dataframe per group
            .groupby(label_col_name)
            # sample nmin observations from each group
            .apply(lambda x: x.sample(nmin,replace=True))
            # recombine the dataframes
            .reset_index(drop=True)
            )

In [75]:
df = downsample(df,"Revue")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56280 entries, 0 to 56279
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Titre        56280 non-null  object
 1   Auteurs      56280 non-null  object
 2   Année        56280 non-null  object
 3   Revue        56280 non-null  object
 4   IndexId      56280 non-null  object
 5   IdRefrences  56280 non-null  object
 6   Abstract     56280 non-null  object
dtypes: object(7)
memory usage: 3.0+ MB


In [76]:
df = df.sample(frac=0.15)

In [77]:
#punctuations
punctuations = string.punctuation
#stopwords
stop_words = list(STOP_WORDS)
def clean_text(
  string : str,
  ponctuation = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~',
  stop_words = stop_words)->str : 

  #URLs
  string = re.sub(r"https?://\S+|www\.\S+",'',string)
  #html 
  string = re.sub(r'<.*?>','',string)
  #Numbers
  string = re.sub(r'[0-9]+','',string) 

  #Remove punctuations
  for x in string.lower():
    if x in punctuations:
      string = string.replace(x,"")
  #To lower
  string = string.lower()
  #Remove stop words
  lemmatizer = WordNetLemmatizer()
  string = ' '.join([lemmatizer.lemmatize(word) for word in string.split() if word not in stop_words])
  #Cleaning the whitespaces
  string = re.sub(r'\s', ' ', string).strip()

  return string
  


In [78]:
from sklearn.feature_extraction.text import CountVectorizer
abstract = df.Abstract.to_list()
abstracts_cleaned = [clean_text(x) for x in abstract]
len(abstracts_cleaned)

8442

In [79]:
print(abstracts_cleaned[0])
print("***************************")
print(df.at[0, 'Abstract'])

multiresolution mesh enable build representation geometric object different level detail lods introduce multiresolution scheme data structure allows separately restore geometry topology mesh refinement process additionallywe use topological criterion geometric criterionas usual literature quickly simplify meshwhat corresponding simplification algorithm adequate realtime application asfor exampleonline computer game
***************************


KeyError: ignored

# **Transformation de données :**

## **Vectorisation (Document - mot )**

In [80]:
vectorizer = CountVectorizer()
DocTerm_abstract = vectorizer.fit_transform(abstracts_cleaned)
print('type of DocTerm_abstract : ', type(DocTerm_abstract))
print("Documents / Mots  : ",DocTerm_abstract.shape)
print(DocTerm_abstract)

DocTerm_abstract = DocTerm_abstract.toarray()
print("******************************************")
print('DocTerm_abstract', type(DocTerm_abstract))


type of DocTerm_abstract :  <class 'scipy.sparse.csr.csr_matrix'>
Documents / Mots  :  (8442, 34013)
  (0, 19148)	2
  (0, 17851)	2
  (0, 9155)	1
  (0, 3242)	1
  (0, 25308)	1
  (0, 11976)	2
  (0, 20310)	1
  (0, 7716)	1
  (0, 16395)	1
  (0, 7472)	1
  (0, 16814)	1
  (0, 14946)	1
  (0, 26508)	1
  (0, 6559)	1
  (0, 29005)	1
  (0, 902)	1
  (0, 27075)	1
  (0, 25583)	1
  (0, 11981)	1
  (0, 31131)	1
  (0, 24938)	1
  (0, 23390)	1
  (0, 374)	1
  (0, 32330)	1
  (0, 31125)	1
  :	:
  (8441, 23074)	2
  (8441, 31383)	2
  (8441, 18485)	1
  (8441, 22802)	1
  (8441, 13696)	2
  (8441, 31724)	1
  (8441, 2192)	3
  (8441, 31362)	3
  (8441, 24556)	1
  (8441, 19494)	1
  (8441, 30539)	1
  (8441, 22858)	1
  (8441, 2581)	1
  (8441, 3038)	1
  (8441, 30861)	5
  (8441, 22068)	1
  (8441, 1075)	1
  (8441, 1079)	1
  (8441, 6869)	1
  (8441, 30069)	1
  (8441, 29897)	1
  (8441, 2838)	1
  (8441, 4456)	1
  (8441, 31190)	9
  (8441, 15807)	1
******************************************
DocTerm_abstract <class 'numpy.ndarray'>


In [ ]:
#Get Words List
print(vectorizer.get_feature_names())


['ability', 'able', 'abstract', 'abstraction', 'acceleration', 'acceptable', 'access', 'accesses', 'accommodate', 'accompanying', 'accomplish', 'account', 'accountand', 'accuracy', 'accuracybut', 'accurate', 'accurately', 'accuratelyand', 'achievable', 'achieve', 'achieved', 'achieves', 'achieving', 'achilles', 'act', 'activation', 'activity', 'activitybased', 'acts', 'actual', 'adapting', 'adaption', 'adaptive', 'added', 'adding', 'additional', 'additionour', 'address', 'addressed', 'addresses', 'addressing', 'adequately', 'adhoc', 'adjacenciesshould', 'adjacency', 'adjacencybased', 'adjust', 'adopting', 'advanced', 'advantage', 'advantages', 'advisor', 'affecting', 'affordable', 'againefficiently', 'aggregating', 'aggressor', 'ahb', 'ai', 'aided', 'algebraic', 'algorithm', 'algorithmas', 'algorithmcalled', 'algorithmknown', 'algorithmnamed', 'algorithms', 'algorithmscalled', 'algorithmsdogmam', 'algorithmsnamed', 'alignment', 'allcritical', 'allocate', 'allotted', 'allour', 'allow', 

## **Exctraction de "Linked entities"**

In [ ]:
import spacy
import en_core_web_sm
from functools import reduce
nlp = spacy.load('en_core_web_sm')
# add pipeline (declared through entry_points in setup.py)
nlp.add_pipe("entityLinker",last=True)

In [ ]:
listEntity =[]
for a,abs in enumerate(abstracts_cleaned):
  doc = nlp(str(abs))
  listInter=[]
  #return all the entities in the whole document
  all_linked_entities = doc._.linkedEntities
  #itereate over sentences and print linked entities
  for sent in doc.sents:
    #print(sent._.linkedEntities)
    for i in range(len(sent._.linkedEntities)):
      entity = sent._.linkedEntities[i].get_id()
      listInter.append(entity)
  listEntity.append(listInter)


In [ ]:
listEntityAll = reduce(operator.concat,listEntity)
print("listEntityAll" , len(listEntityAll))
listEntityAllunique = np.unique(np.asarray(listEntityAll))
print("listEntityAllunique" , len(listEntityAllunique))


listEntityAll 1682
listEntityAllunique 566


## **BOW**

In [ ]:
dictindex = dict(zip(listEntityAllunique,np.arange(len(listEntityAllunique))))
entityEmbedding = np.zeros((len(abstract),len(listEntityAllunique)))
for a in range(len(abstract)):
  lista = listEntity[a]
  indexEn = values = np.asanyarray(list(map(dictindex.get,lista)))
  entityEmbedding[a,indexEn]=1

## **BERT**

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens')
sentence_embedding = model.encode(abstract)

In [ ]:
sentence_embedding

array([[-0.90498275,  0.20419133,  0.54003567, ...,  0.1595102 ,
        -0.02890011,  0.21780182],
       [-0.84162533,  0.37580818,  0.31556702, ..., -0.15957555,
        -0.1251584 ,  0.36724833],
       [-0.0920645 ,  0.60817575,  0.5032281 , ..., -0.46368083,
        -1.1678922 ,  0.628837  ],
       ...,
       [-0.8522707 ,  0.6357087 ,  0.32293302, ..., -0.90342045,
        -0.16440001,  0.16181709],
       [-0.15328439,  0.30836326,  1.0114954 , ..., -0.23881237,
        -0.5476271 ,  0.27469596],
       [-0.00146912,  0.23846017,  0.8166513 , ..., -0.25199023,
        -1.2100103 ,  0.4056557 ]], dtype=float32)

# **Clustering**

In [81]:
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score

K=28
labels_all, u  = pd.factorize(df['Revue'])

kmeans = KMeans(n_clusters=K, random_state=0,n_init=5).fit(DocTerm_abstract)
y_pred1 = kmeans.labels_
print(y_pred1)
print('Bow representation clustering  accuracy : ',accuracy_score(labels_all,y_pred1))

[ 1 20 23 ...  5  2  1]
Bow representation clustering  accuracy :  0.045249940772328835


In [61]:
labels_all, xnxx = pd.factorize(df['Revue'])
print(labels_all)
print(xnxx)

[ 0  0  0 ... 27 27 27]
Index(['DAC', 'DAGM-Symposium', 'DATA BASE', 'Data Min. Knowl. Discov.',
       'Des. Codes Cryptography', 'Discrete Applied Mathematics',
       'Distributed Computing', 'Electr. Notes Theor. Comput. Sci.', 'GECCO',
       'GECCO (Companion)', 'GECCO Workshops', 'ICC', 'ICCAD', 'ICCBR', 'ICCE',
       'ICCSA (1)', 'ICCSA (2)', 'ICCSA (3)', 'ICCV', 'ICIP', 'IJCAI',
       'IJCAI (1)', 'Journal of Systems and Software', 'Knowl. Inf. Syst.',
       'SIGMOD Conference', 'SIGMOD Record', 'SMC', 'STOC'],
      dtype='object')


In [62]:
np.unique(labels_all)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27])

In [ ]:
kmeans = KMeans(n_clusters=K, random_state=0,n_init=5).fit(entityEmbedding)
#kmeans = KMeans(n_clusters=K, random_state=0,n_init=5).fit(mat)

y_pred2 = kmeans.labels_
print(y_pred2)
#print('Entity  representation clustering  accuracy : ',accuracy_score(labels_all,y_pred2))

In [ ]:
kmeans = KMeans(n_clusters=K, random_state=0,n_init=5).fit(sentence_embedding)
y_pred3 = kmeans.labels_
print(y_pred3)
#print('Sentence  representation clustering  accuracy : ',accuracy_score(labels_all,y_pred3))

In [ ]:
import numpy as np
import Cluster_Ensembles as CE
import functools
from functools import reduce
clustering_1 = y_pred1
clustering_2 = y_pred2
clustering_3 = y_pred3
cluster_runs = np.array([clustering_1,clustering_2,clustering_3])
print("cluster_runs" , cluster_runs)
#consensus_clustering_labels = CE.cluster_ensembles(cluster_runs,verbose = True,N_clusters_max = 3)
#print("consensus_clustering_labels",consensus_clustering_labels)

cluster_runs [[2 0 1 1 1 2 2 2 2 2 0 2 2 2 0 2 1 2 0 1 2 1 1 2 2 2 1 2 1 1 0 2 2 2 2 2
  2 1 2 2 2 2 2 0 0 2 1 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0
  1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 2 1 2 2 2 1 1]
 [2 2 0 0 1 1 1 2 1 1 1 1 1 2 1 0 1 1 0 2 0 1 2 1 2 1 0 2 1 1 2 2 1 1 1 0
  1 1 1 0 1 1 2 2 1 0 2 2 2 2 2 2 1 1 1 2 2 1 1 1 2 0 1 1 0 1 1 1 1 1 1 1
  1 0 1 1 1 0 2 1 1 1 1 0 0 1 0 1 1 1 0 1 1 0 1 2 1 1 1 1]
 [1 1 1 1 1 1 1 1 2 1 1 0 1 0 1 0 2 2 2 1 1 1 1 0 0 0 1 0 1 0 1 0 2 1 2 1
  1 2 1 0 2 0 0 0 1 0 1 0 0 1 1 1 1 1 1 0 0 0 2 2 0 0 0 1 1 1 1 1 1 2 2 1
  1 0 2 0 2 0 1 1 2 2 2 0 1 1 0 2 1 0 1 0 0 1 0 1 2 1 1 1]]


NameError: ignored

In [ ]:
#from sklearn.metrics import accuracy_score
#print('Sentence  representation clustering  accuracy : ',accuracy_score(labels_all  ,consensus_clustering_labels ))

Sentence  representation clustering  accuracy :  0.2303628847037207
